# Results combination and analysis

In [2]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | Cape Town |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | Johan W. Joubert |
| Contact: | elias@wastelabs.co |

In [55]:
import csv
import glob
import io
import logging
import os
import subprocess
from datetime import datetime

import plotly.express as px
import plotly.io as pio
import requests

pio.renderers.default = "iframe"

import re

import boto3
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import yaml
from GPSOdyssey import Kepler
from IPython.core.interactiveshell import InteractiveShell

# Ipython configs
from IPython.display import HTML, display
from shapely.geometry import LineString

display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

## Find partition keys

In [56]:
df_producer_geo_all = catalog.load("syn_pop_scenarios_local")
waste_all = catalog.load("waste_gen_scenarios_local")
results_all = catalog.load("resource_summary_table")
keys = list(df_producer_geo_all)
keys = [x for x in keys if x[0] != "."]

2022-05-30 23:56:20,930 - kedro.io.data_catalog - INFO - Loading data from `syn_pop_scenarios_local` (PartitionedDataSet)...
2022-05-30 23:56:20,945 - kedro.io.data_catalog - INFO - Loading data from `waste_gen_scenarios_local` (PartitionedDataSet)...
2022-05-30 23:56:20,957 - kedro.io.data_catalog - INFO - Loading data from `resource_summary_table` (PartitionedDataSet)...


## Extract and summaries

In [57]:
i = 0

In [58]:
all_results = []
for key in keys:
    results = results_all[key]()
    results = results.loc[results["Unnamed: 0"] == "Total"]
    results["scenario"] = key
    all_results.append(results)
all_results = pd.concat(all_results)
all_results = all_results.drop(columns=["Unnamed: 0", "Collection day"])
all_results[
    ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
] = all_results[
    ["Tons disposed at @ Offload 3", "Tons disposed at @ Offload 2"]
].fillna(
    0
)

## Store results

In [59]:
%reload_kedro
catalog.save("resource_scenario_results", all_results)

2022-05-30 23:56:23,570 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-05-30 23:56:23,643 - root - INFO - ** Kedro project Demand estimation and waste collection routing optimisation for the City of Cape Town
2022-05-30 23:56:23,644 - root - INFO - Defined global variable `context`, `session` and `catalog`
2022-05-30 23:56:23,653 - root - INFO - Registered line magic `run_viz`
2022-05-30 23:56:23,655 - kedro.io.data_catalog - INFO - Saving data to `resource_scenario_results` (CSVDataSet)...


## Early graphs

In [91]:
fig = px.histogram(all_results, x="Demand collected (ton)")
fig

fig = px.histogram(all_results, x="Units served")
fig

In [92]:
fig = px.histogram(all_results, x="Route duration (h)")
fig

In [77]:
fig = px.histogram(all_results, x="Total route distance (km)")
fig

In [78]:
fig = px.scatter(all_results, x="Units served", y="Route duration (h)")
fig
fig = px.scatter(all_results, x="Units served", y="Total route distance (km)")
fig

In [82]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Route duration (h)")
fig
fig = px.scatter(all_results, x="Demand collected (ton)", y="Total route distance (km)")
fig

In [83]:
fig = px.scatter(all_results, x="Units served", y="Demand collected (ton)")
fig

In [84]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Total route distance (km)", color="Units served")
fig

In [85]:
fig = px.scatter(all_results, x="Demand collected (ton)", y="Route duration (h)", color="Units served")
fig

In [86]:
from pandas_profiling import ProfileReport

2022-05-31 00:06:39,906 - visions.backends - INFO - Pandas backend loaded 1.4.2
2022-05-31 00:06:39,916 - visions.backends - INFO - Numpy backend loaded 1.22.3
2022-05-31 00:06:39,918 - visions.backends - INFO - Pyspark backend NOT loaded
2022-05-31 00:06:39,919 - visions.backends - INFO - Python backend loaded


In [88]:
profile = ProfileReport(all_results, title="Waste generation scenario analysis")

In [89]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/missingno/missingno.py:72: MatplotlibDeprecationWarning:

The 'b' parameter of grid() has been renamed 'visible' since Matplotlib 3.5; support for the old name will be dropped two minor releases later.



Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

2022-05-31 00:08:04,033 - root - ERROR - Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_18063/2130615311.py", line 1, in <cell line: 1>
    profile.to_widgets()
  File "/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/pandas_profiling/profile_report.py", line 414, in to_widgets
    display(self.widgets)
  File "/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/pandas_profiling/profile_report.py", line 197, in widgets
    self._widgets = self._render_widgets()
  File "/Users/ejwillemse/opt/anaconda3/envs/python_RD/lib/python3.8/site-packages/pandas_profiling/profile_report.py", line 323, in _render

TypeError: object of type 'NoneType' has no len()

In [90]:
profile.to_file("../data/08_reporting/Waste generation scenario analysis.html")

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]